In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

StatementMeta(, f752fc3b-1cce-4591-b274-b2c32dcb6275, 3, Finished, Available, Finished)

In [2]:
sch_broker = StructType([
    StructField("id_brk", IntegerType(), False),
    StructField("name_brk", StringType(), True),
    StructField("region_brk", StringType(), True),
    StructField("email_brk", StringType(), True)
])

sch_campania = StructType([
    StructField("id_cam", IntegerType(), False),
    StructField("channel_cam", StringType(), True),
    StructField("name_cam", StringType(), True),
    StructField("date_start_cam", DateType(), True),
    StructField("date_end_cam", DateType(), True),
    StructField("cost_cam", DoubleType(), True)
])

sch_cliente = StructType([
    StructField("id_cons", IntegerType(), False),
    StructField("name_cons", StringType(), True),
    StructField("lastname_cons", StringType(), True),
    StructField("email_cons", StringType(), True),
    StructField("region_cons", StringType(), True)
])

sch_interesado = StructType([
    StructField("id_int", IntegerType(), False),
    StructField("id_cons", IntegerType(), False),
    StructField("id_prt", IntegerType(), True),
    StructField("id_cam", IntegerType(), True),
    StructField("fecha_emi", DateType(), True),
    StructField("half", StringType(), True)
])

sch_proyecto = StructType([
    StructField("id_pry", IntegerType(), False),
    StructField("name_pry", StringType(), True),
    StructField("city_pry", StringType(), True),
    StructField("region_pry", StringType(), True),
    StructField("date_start_pry", DateType(), True),
    StructField("state_pry", StringType(), True)
])

sch_propiedad = StructType([
    StructField("id_prop", IntegerType(), False),
    StructField("id_pry", IntegerType(), False),
    StructField("type", StringType(), True),
    StructField("size", DoubleType(), True),
    StructField("rooms", IntegerType(), True),
    StructField("bathrooms", IntegerType(), True),
    StructField("price", DoubleType(), True),
    StructField("state", StringType(), True)
])

sch_ventas = StructType([
    StructField("id_sale", IntegerType(), False),
    StructField("id_prop", IntegerType(), False),
    StructField("id_cons", IntegerType(), True),
    StructField("id_brk", IntegerType(), True),
    StructField("date_sale", DateType(), True),
    StructField("price_sale", DoubleType(), True)
])

print("Definicion de esquemas exitoso")

StatementMeta(, f752fc3b-1cce-4591-b274-b2c32dcb6275, 4, Finished, Available, Finished)

Definicion de esquemas exitoso


In [3]:
try:

    df_brokers = spark.read.format("csv") \
        .option("header", "true") \
        .schema(sch_broker) \
        .load("Files/bronze/brokers.csv")
    

    df_campaigns = spark.read.format("csv") \
        .option("header", "true") \
        .schema(sch_campania) \
        .load("Files/bronze/campaigns.csv")
    

    df_consumer = spark.read.format("csv") \
        .option("header", "true") \
        .schema(sch_cliente) \
        .load("Files/bronze/clients.csv")
    

    df_interested = spark.read.format("csv") \
        .option("header", "true") \
        .schema(sch_interesado) \
        .load("Files/bronze/leads.csv")
    

    df_proyects = spark.read.format("csv") \
        .option("header", "true") \
        .schema(sch_proyecto) \
        .load("Files/bronze/projects.csv")
    

    df_properties = spark.read.format("csv") \
        .option("header", "true") \
        .schema(sch_propiedad) \
        .load("Files/bronze/properties.csv")


    df_sales = spark.read.format("csv") \
        .option("header", "true") \
        .schema(sch_ventas) \
        .load("Files/bronze/sales.csv")
    print("Lectura exitosa")
except Exception as e:
    print(f"Error: {str(e)}")

StatementMeta(, f752fc3b-1cce-4591-b274-b2c32dcb6275, 5, Finished, Available, Finished)

Lectura exitosa


In [4]:
df_brokers.write.format("delta").mode("overwrite").save("Files/silver/brokers")
df_campaigns.write.format("delta").mode("overwrite").save("Files/silver/campaigns")
df_consumer.write.format("delta").mode("overwrite").save("Files/silver/costumers")
df_interested.write.format("delta").mode("overwrite").save("Files/silver/interested")
df_proyects.write.format("delta").mode("overwrite").save("Files/silver/proyects")
df_properties.write.format("delta").mode("overwrite").save("Files/silver/properties")
df_sales.write.format("delta").mode("overwrite").save("Files/silver/sales")

StatementMeta(, f752fc3b-1cce-4591-b274-b2c32dcb6275, 6, Finished, Available, Finished)

In [5]:
from pyspark.sql.functions import col

df_brokers_gold = df_brokers.filter(
    col("id_brk").isNotNull() &
    col("name_brk").isNotNull() &
    col("region_brk").isNotNull()
)


df_campaigns_gold = df_campaigns.filter(
    col("id_cam").isNotNull() &
    col("channel_cam").isNotNull() &
    col("name_cam").isNotNull() &
    col("date_start_cam").isNotNull()
)


df_clients_gold = df_consumer.filter(
    col("id_cons").isNotNull() &
    col("name_cons").isNotNull() &
    col("lastname_cons").isNotNull()
)


df_interested_gold = df_interested.filter(
    col("id_int").isNotNull() &
    col("id_cons").isNotNull() &
    col("fecha_emi").isNotNull()
)


df_projects_gold = df_proyects.filter(
    col("id_pry").isNotNull() &
    col("name_pry").isNotNull() &
    col("city_pry").isNotNull() &
    col("region_pry").isNotNull()
)

df_properties_gold = df_properties.filter(
    col("id_prop").isNotNull() &
    col("id_pry").isNotNull() &
    col("type").isNotNull() &
    col("size").isNotNull() &
    col("price").isNotNull()
)

df_sales_gold = df_sales.filter(
    col("id_sale").isNotNull() &
    col("id_prop").isNotNull() &
    col("date_sale").isNotNull() &
    col("price_sale").isNotNull()
)


StatementMeta(, f752fc3b-1cce-4591-b274-b2c32dcb6275, 7, Finished, Available, Finished)

In [6]:
df_brokers_gold.write.format("delta").mode("overwrite").save("Files/gold/brokers")
df_campaigns_gold.write.format("delta").mode("overwrite").save("Files/gold/campaigns")
df_clients_gold.write.format("delta").mode("overwrite").save("Files/gold/clients")
df_interested_gold.write.format("delta").mode("overwrite").save("Files/gold/interested")
df_projects_gold.write.format("delta").mode("overwrite").save("Files/gold/projects")
df_properties_gold.write.format("delta").mode("overwrite").save("Files/gold/properties")



df_sales_gold.write.format("delta").mode("overwrite").save("Files/gold/sales")


StatementMeta(, f752fc3b-1cce-4591-b274-b2c32dcb6275, 8, Finished, Available, Finished)

In [7]:

try:
    df_brokers_gold.createOrReplaceTempView("dim_brokers")
    df_campaigns_gold.createOrReplaceTempView("dim_campaigns")
    df_clients_gold.createOrReplaceTempView("dim_clients")
    df_interested_gold.createOrReplaceTempView("fact_interested")
    df_projects_gold.createOrReplaceTempView("dim_projects")
    df_properties_gold.createOrReplaceTempView("dim_properties")
    df_sales_gold.createOrReplaceTempView("fact_sales")

    tables = spark.sql("SHOW TABLES").collect()

    df_brokers_gold.write.format("delta").mode("overwrite").saveAsTable("dim_brokers")
    df_campaigns_gold.write.format("delta").mode("overwrite").saveAsTable("dim_campaigns")
    df_clients_gold.write.format("delta").mode("overwrite").saveAsTable("dim_clients")
    df_interested_gold.write.format("delta").mode("overwrite").saveAsTable("fact_interested")
    df_projects_gold.write.format("delta").mode("overwrite").saveAsTable("dim_projects")
    df_properties_gold.write.format("delta").mode("overwrite").saveAsTable("dim_properties")
    df_sales_gold.write.format("delta").mode("overwrite").saveAsTable("fact_sales")
    
    print("Tablas creadas exitosamente")
except Exception as e:
    print(f"Error: {str(e)}")

StatementMeta(, f752fc3b-1cce-4591-b274-b2c32dcb6275, 9, Finished, Available, Finished)

Tablas creadas exitosamente
